# Preprocesamiento

## Importar librerias

### Modelo inicial sin GridSearch

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from  xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_excel("Tabla Trabajo Grupal N°2.xlsx", sheet_name= "Desarrollo")
df.head()

,Id_Cliente,Edad,Nivel_Educacional,Años_Trabajando,Ingresos,Deuda_Comercial,Deuda_Credito,Otras_Deudas,Ratio_Ingresos_Deudas,Default
0,1,34,SupInc,11,68,11.9,3.37,4.73,0.29,1
1,2,43,Med,3,53,9.8,1.51,3.69,0.28,0
2,3,36,SupInc,7,118,11.2,8.48,4.73,0.21,0
3,4,46,Bas,11,61,10.4,3.44,2.90,0.27,1
4,5,29,Med,0,21,12.4,0.38,2.22,0.71,0


In [16]:
def limpiar_columnas(df):
    df.drop("Id_Cliente",  errors= "ignore", inplace = True)
    return df.drop("Default", axis = 1), df["Default"]

df_x, df_y = limpiar_columnas(df)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state= 7, test_size=0.3)
var_num = df_x.select_dtypes("number").columns
var_cat = df_x.select_dtypes("object").columns
pip_cat = Pipeline(steps = [
                            ("ohe", OneHotEncoder())
                            ]
                    )

pp = ColumnTransformer(transformers= 
                       [("ohe", pip_cat, var_cat)],
                       remainder='passthrough')

model_pipe = Pipeline(steps =
                      [
                        ("pp", pp)
                       ,("modelo", LogisticRegression())
                          
                      ]
                    )

model_pipe.fit(x_train, y_train)

y_predicto = model_pipe.predict(x_test)

acc_RL = accuracy_score(y_test, y_predicto)
print(acc_RL)


0.755867278122471


c:\Users\hemer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Modelo con GridSearch

In [ ]:
param_grid = {
    "modelo__C": [0.1, 1,2,3,4],
    "modelo__penalty": ["l1", "l2"],
    "modelo__solver": ["liblinear"]
}

busqueda = GridSearchCV(
    estimator = model_pipe,
    param_grid= param_grid,
    cv = 5,
    n_jobs= -1,
    scoring= "accuracy",
    
)



In [10]:
df.head()

,Id_Cliente,Edad,Nivel_Educacional,Años_Trabajando,Ingresos,Deuda_Comercial,Deuda_Credito,Otras_Deudas,Ratio_Ingresos_Deudas,Default
0,1,34,SupInc,11,68,11.9,3.37,4.73,0.29,1
1,2,43,Med,3,53,9.8,1.51,3.69,0.28,0
2,3,36,SupInc,7,118,11.2,8.48,4.73,0.21,0
3,4,46,Bas,11,61,10.4,3.44,2.90,0.27,1
4,5,29,Med,0,21,12.4,0.38,2.22,0.71,0


In [19]:
df = pd.read_excel("Tabla Trabajo Grupal N°2.xlsx", sheet_name= "Desarrollo")


def limpiar_columnas(df):
    df.drop("Id_Cliente",  errors= "ignore", inplace = True)
    return df.drop("Default", axis = 1), df["Default"]

df_x, df_y = limpiar_columnas(df)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state= 7, test_size=0.3)
var_num = df_x.select_dtypes("number").columns
var_cat = df_x.select_dtypes("object").columns
pip_cat = Pipeline(steps = [
                            ("ohe", OneHotEncoder())
                            ]
                    )

pp = ColumnTransformer(transformers= 
                       [("ohe", pip_cat, var_cat)],remainder='passthrough')

model_pipe = Pipeline(steps =
                      [
                        ("pp", pp)
                       ,("modelo", LogisticRegression())
                          
                      ]
                    )


param_grid = [
    # COMPETIDOR 1: Regresión Logística
    {
        'modelo': [LogisticRegression(max_iter=1000)],
        'modelo__C': [0.1, 1, 10],
        'modelo__solver': ['liblinear']
    },
    # COMPETIDOR 2: Random Forest
    {
        'modelo': [RandomForestClassifier( )],
        'modelo__n_estimators': [100, 200],
        'modelo__max_depth': [1,2,3,4]
    },
    # COMPETIDOR 3: XGBoost
    {
        'modelo': [XGBClassifier(random_state=7, use_label_encoder=False, eval_metric='logloss')],
        #'modelo__learning_rate': [0.01, 0.1],
        #'modelo__n_estimators': [1,2,3,4]
    }
]

busqueda_competitiva = GridSearchCV(
    estimator=model_pipe,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

busqueda_competitiva.fit(x_train, y_train)


# --- RESULTADOS ---
print(f"El modelo ganador es: {busqueda_competitiva.best_params_['modelo']}")
print(f"Mejor Accuracy (CV): {busqueda_competitiva.best_score_:.4f}")


print(".......................")

y_pred = busqueda_competitiva.predict(x_test)
print(accuracy_score(y_test, y_pred))



resultados = pd.DataFrame(busqueda_competitiva.cv_results_)

def nombre_modelo(est):
    return est.__class__.__name__

resultados["modelo_nombre"] = resultados["param_modelo"].apply(nombre_modelo)

acc_cv_por_modelo = (
    resultados
    .groupby("modelo_nombre")["mean_test_score"]
    .max()
    .sort_values(ascending=False)
)

print("Accuracy CV por modelo:")
print(acc_cv_por_modelo)



Fitting 5 folds for each of 12 candidates, totalling 60 fits
El modelo ganador es: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)
Mejor Accuracy (CV): 0.9970
.......................
1.0
Accuracy CV por modelo:
modelo_nombre
XGBClassifier             0.996994
RandomFo

c:\Users\hemer\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [15:36:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
df_test = pd.read_excel("Tabla Trabajo Grupal N°2.xlsx", sheet_name= "Validación")


df_validacion, _ = limpiar_columnas(df_test)